## word2vec을 활용한 모델 구현

In [28]:
import pandas as pd
import numpy as np

In [3]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

sentences = []
for review in reviews:
    sentences.append(review.split())

In [10]:
num_features = 300   # 워드 벡터 특징값 수
min_word_count = 40  # 단어에 대한 최소 빈도 수
num_workers = 4      # 프로세스 개수
context = 10         # 컨텍스트 윈도우 크기 
downsampling = 1e-3  # 다운 샘플링 비율

- num_features: 각 단어에 대해 임베딩된 벡터의 차원을 정한다
- min_word_count: 모델에 의미 있는 단어를 가지고 학습하기 위해 적은 빈도 수의 단어들은 학습하지 않는다
- num_workers: 모델 학습 시 학습을 위한 프로세스 개수를 지정한다.
- context: word2vec을 수행하기 위한 컨텍스트 윈도우 크기를 지정한다.
- downsampling: word2vec 학습을 수행할 때 빠른 학습을 위해 정답 단어 라벨에 대한 다운 샘플링 비율을 지정한다. 보통 0.001이 좋은 성능을 낸다고 한다.

In [17]:
!pip install gensim

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


진행 상황 확인

In [18]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

학습

In [19]:
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences,
                         workers=num_workers,
                         size=num_features,
                         min_count = min_word_count,
                         window = context,
                         sample = downsampling)

2019-05-09 16:36:32,534 : INFO : 'pattern' package not found; tag filters are not available for English
2019-05-09 16:36:32,547 : INFO : collecting all words and their counts
2019-05-09 16:36:32,548 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2019-05-09 16:36:32,924 : INFO : PROGRESS: at sentence #10000, processed 1221909 words, keeping 55812 word types
2019-05-09 16:36:33,290 : INFO : PROGRESS: at sentence #20000, processed 2429834 words, keeping 74689 word types
2019-05-09 16:36:33,471 : INFO : collected 82107 word types from a corpus of 3029622 raw words and 25000 sentences
2019-05-09 16:36:33,472 : INFO : Loading a fresh vocabulary
2019-05-09 16:36:33,642 : INFO : effective_min_count=40 retains 8132 unique words (9% of original 82107, drops 73975)
2019-05-09 16:36:33,643 : INFO : effective_min_count=40 leaves 2648221 word corpus (87% of original 3029622, drops 381401)
2019-05-09 16:36:33,682 : INFO : deleting the raw counts dictionary of 82107 items
2019-05-09 16:36:33,686 : INFO : sample=0.001 downsamples 29 most-common words
2019-05-09 16:36:33,687 : INFO : downsampling leaves estimated 2490621 word corpus (94.0% of prior 2648221)
2019-05-09 16:36:33,729 : INFO : estimated required memory for 8132 words and 300 dimens

모델 저장 해놓기

In [20]:
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-05-09 16:41:03,924 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2019-05-09 16:41:03,926 : INFO : not storing attribute vectors_norm
2019-05-09 16:41:03,929 : INFO : not storing attribute cum_table
2019-05-09 16:41:04,246 : INFO : saved 300features_40minwords_10context


In [31]:
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype = np.float32)
    
    num_words = 0
    # 어휘사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words += 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
            
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

이렇게 문장에 특징값을 만들 수 있는 함수를 구현했다면 이제 앞에서 정의한 함수를 사용해 전체 리뷰에 대해 각 리뷰의 평균 벡터를 구하는 함수를 정의한다.

In [25]:
def get_dataset(reviews, model, num_features):
    dataset = list()
    
    for s in reviews:
        dataset.append(get_features(s, model, num_features))
        
    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [34]:
test_data_vecs = get_dataset(sentences, model, num_features)

C:\Users\zlslsp54\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


만들어진 데이터를 가지고 학습 데이터와 검증 데이터를 나눠보자

In [42]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size = TEST_SPLIT, random_state = RANDOM_SEED)

여기까지 word2vec 완성

### 모델 선언 및 학습

로지스틱 모델

In [46]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(X_train, y_train)

C:\Users\zlslsp54\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

검증 데이터셋을 이용한 성능 평가

In [48]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

Accuracy: 0.866400


TF-IDF와 별로 차이가 나지 않는다.

### 데이터 캐글 제출

In [ ]:
TESt